In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# ------------------ Random Forest Classifier ------------------
rf_param_grid = {
    'n_estimators': [50, 100, 200],     # number of trees
    'max_depth': [None, 10, 20],        # tree depth
    'criterion': ['gini', 'entropy']    # split measure
}
grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# fitting the model for grid search
grid.fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [12]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [13]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [14]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'n_estimators': 200}
The f1_macro: 0.8545314767655193
The confusion Matrix:
 [[230  27]
 [ 31 112]]
The report:
               precision    recall  f1-score   support

           0       0.88      0.89      0.89       257
           1       0.81      0.78      0.79       143

    accuracy                           0.85       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.85      0.85      0.85       400



In [15]:
# Save Best Model
import pickle
filename = "RFC_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [16]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.257582,0.010385,0.013848,0.003225,gini,None,50,"{'criterion': 'gini', 'max_depth': None, 'n_es...",0.8000,0.9375,0.8750,0.8375,0.8125,0.8525,0.049624,3
1,0.473143,0.062731,0.021161,0.003794,gini,None,100,"{'criterion': 'gini', 'max_depth': None, 'n_es...",0.8000,0.9375,0.8750,0.8375,0.8000,0.8500,0.051841,6
2,0.506776,0.027776,0.026218,0.001280,gini,None,200,"{'criterion': 'gini', 'max_depth': None, 'n_es...",0.8000,0.9250,0.8875,0.8375,0.8250,0.8550,0.045139,1
3,0.124971,0.002680,0.008188,0.000237,gini,10,50,"{'criterion': 'gini', 'max_depth': 10, 'n_esti...",0.7875,0.9375,0.8750,0.8375,0.8125,0.8500,0.052440,6
4,0.247143,0.006403,0.015090,0.001160,gini,10,100,"{'criterion': 'gini', 'max_depth': 10, 'n_esti...",0.7875,0.9375,0.8750,0.8375,0.8000,0.8475,0.054429,10
